<a href="https://colab.research.google.com/drive/1YzAzsXJYqrU2wJ3waBiaLAbOJJCuIoZm?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [2]:
# Install required libraries
!pip install -q unsloth accelerate bitsandbytes datasets einops xformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 506.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Load LLaMA-2-7B in 4-bit using Unsloth
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-2-7b-bnb-4bit",
    max_seq_length=512,
    dtype=torch.float16,
    load_in_4bit=True
)

tokenizer.pad_token = tokenizer.eos_token

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [4]:
# Enable LoRA for fine-tuning
FastLanguageModel.for_training(model, use_gradient_checkpointing=True)
model = FastLanguageModel.get_peft_model(model)


Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
from datasets import load_dataset

# Load smaller dataset
dataset = load_dataset("xsum", split="train[:5000]")

def format_example(example):
    return {
        "text": f"### Article:\n{example['document']}\n\n### Summary:\n{example['summary']}"
    }

dataset = dataset.map(format_example)

def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

# Trainer setup
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama_unsloth_summarizer",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,
    max_steps=50,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Fast training
trainer.train()

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 39,976,960/7,000,000,000 (0.57% trained)


Step,Training Loss
10,1.747700
20,1.674000
30,1.680900
40,1.690600
50,1.750700


TrainOutput(global_step=50, training_loss=1.7087652587890625, metrics={'train_runtime': 270.3463, 'train_samples_per_second': 0.74, 'train_steps_per_second': 0.185, 'total_flos': 4084113761894400.0, 'train_loss': 1.7087652587890625, 'epoch': 0.04})

In [13]:
# Save trained LoRA adapter
model.save_pretrained("tinyllama_unsloth_summarizer_lora")
tokenizer.save_pretrained("tinyllama_unsloth_summarizer_lora")

('tinyllama_unsloth_summarizer_lora/tokenizer_config.json',
 'tinyllama_unsloth_summarizer_lora/special_tokens_map.json',
 'tinyllama_unsloth_summarizer_lora/tokenizer.model',
 'tinyllama_unsloth_summarizer_lora/added_tokens.json',
 'tinyllama_unsloth_summarizer_lora/tokenizer.json')

In [15]:
# Test model with new unseen articles

model.eval()

prompt = """### Instruction:
Summarize the article below in 1–2 sentences.

### Article:
Scientists exploring the remote jungles of the Amazon have discovered a previously unknown species of bird.
The newly identified bird is notable for its vibrant, multicolored feathers that shimmer under sunlight.
Unlike other related species, this bird produces a unique and melodious call, making it easy to distinguish.
Researchers believe the bird has evolved in isolation, adapting to the dense, humid environment of the Amazon.
Its discovery highlights the incredible biodiversity that remains hidden within the world's rainforests.
Scientists have already begun detailed studies of its behavior, diet, and mating patterns.
Preliminary observations suggest it feeds primarily on exotic fruits and small insects.
Conservationists are stressing the importance of preserving the bird's fragile habitat from deforestation.
This discovery may also shed light on evolutionary patterns among tropical bird species.
The scientific community has hailed the find as a major contribution to our understanding of avian diversity.

### Summary:"""

inputs2 = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs2 = model.generate(
        **inputs2,
        max_new_tokens=80,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

print(tokenizer.decode(outputs2[0], skip_special_tokens=True))

### Instruction:
Summarize the article below in 1–2 sentences.

### Article:
Scientists exploring the remote jungles of the Amazon have discovered a previously unknown species of bird.
The newly identified bird is notable for its vibrant, multicolored feathers that shimmer under sunlight.
Unlike other related species, this bird produces a unique and melodious call, making it easy to distinguish.
Researchers believe the bird has evolved in isolation, adapting to the dense, humid environment of the Amazon.
Its discovery highlights the incredible biodiversity that remains hidden within the world's rainforests.
Scientists have already begun detailed studies of its behavior, diet, and mating patterns.
Preliminary observations suggest it feeds primarily on exotic fruits and small insects.
Conservationists are stressing the importance of preserving the bird's fragile habitat from deforestation.
This discovery may also shed light on evolutionary patterns among tropical bird species.
The scient